<a href="https://colab.research.google.com/github/Pranay898/AI-Image-Generator-using-React.js/blob/main/Crop_Yield_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd
import os
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as py
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [35]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import os
%matplotlib inline

In [36]:
import plotly.express as px

In [37]:
yield_df = pd.read_csv('/content/yield_df.csv')

In [38]:
yield_df

,Unnamed: 0,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,0,Albania,Maize,1990,36613,1485.0,121.00,16.37
1,1,Albania,Potatoes,1990,66667,1485.0,121.00,16.37
2,2,Albania,"Rice, paddy",1990,23333,1485.0,121.00,16.37
3,3,Albania,Sorghum,1990,12500,1485.0,121.00,16.37
4,4,Albania,Soybeans,1990,7000,1485.0,121.00,16.37
...,...,...,...,...,...,...,...,...
28237,28237,Zimbabwe,"Rice, paddy",2013,22581,657.0,2550.07,19.76
28238,28238,Zimbabwe,Sorghum,2013,3066,657.0,2550.07,19.76
28239,28239,Zimbabwe,Soybeans,2013,13142,657.0,2550.07,19.76
28240,28240,Zimbabwe,Sweet potatoes,2013,22222,657.0,2550.07,19.76


In [39]:
yield_df.groupby(['Item','Year'], sort = True)['hg/ha_yield'].sum().nlargest(1)

Item      Year
Potatoes  2013    44761561
Name: hg/ha_yield, dtype: int64

In [40]:
yield_df.head()

,Unnamed: 0,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,0,Albania,Maize,1990,36613,1485.0,121.0,16.37
1,1,Albania,Potatoes,1990,66667,1485.0,121.0,16.37
2,2,Albania,"Rice, paddy",1990,23333,1485.0,121.0,16.37
3,3,Albania,Sorghum,1990,12500,1485.0,121.0,16.37
4,4,Albania,Soybeans,1990,7000,1485.0,121.0,16.37


In [41]:
#highest yield
yield_df.groupby(['Area'], sort = True)[['hg/ha_yield']].sum().nlargest(10, 'hg/ha_yield')

,hg/ha_yield
Area,
India,327420324
Brazil,167550306
Mexico,130788528
Japan,124470912
Australia,109111062
Pakistan,73897434
Indonesia,69193506
United Kingdom,55419990
Turkey,52263950


In [42]:
yield_df.groupby(['Area','Item'], sort = True)[['hg/ha_yield']].sum().nlargest(10, 'hg/ha_yield')

hg/ha_yield
Area           Item                       
India          Cassava           142810624
               Potatoes           92122514
Brazil         Potatoes           49602168
United Kingdom Potatoes           46705145
Australia      Potatoes           45670386
India          Sweet potatoes     44439538
Japan          Potatoes           42918726
Mexico         Potatoes           42053880
               Sweet potatoes     35808592
Australia      Sweet potatoes     35550294

In [43]:
yield_df = yield_df.rename({'hg/ha_yield': 'Yield'}, axis='columns') #renaming column
yield_df = yield_df.rename({'Unnamed: 0': 'ID'}, axis='columns')
yield_df = yield_df.rename({'pesticides_tonnes': 'Pesticide_Tons'}, axis='columns')
yield_df = yield_df.rename({'average_rain_fall_mm_per_year': 'Avg_rainfall_mm_per_year'}, axis='columns')
yield_df = yield_df.rename({'avg_temp': 'Avg_temp'}, axis='columns')
yield_df = yield_df.rename({'Area': 'Location'}, axis='columns')

In [44]:
yieldline = px.line(
    yield_df.groupby(['Year'])[['Yield','Pesticide_Tons']].mean(),
    labels={'value': 'Yield and Pesticide_Tons'},
    template='plotly_dark'
)
yieldline.update_layout(
    showlegend=True,
    font={'size': 16},
    title={'text': '<b>Year vs. Yield and Pesticide Use</b>', 'font': {'size': 20}}
)


In [45]:
averagerainfallline = px.line(yield_df.groupby(['Year'])['Avg_rainfall_mm_per_year'].mean(),
               labels = {'value' : 'Avg_rainfall_mm_per_year'}, template = 'simple_white')
averagerainfallline.update_layout(
    showlegend=False,
    font={'size': 16},
    title={'text': '<b>Year vs. Average Rainfall</b>', 'font': {'size': 20}}
)

In [46]:
yield_df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,28242.0,14120.500000,8152.907488,0.00,7060.2500,14120.50,21180.75,28241.00
Year,28242.0,2001.544296,7.051905,1990.00,1995.0000,2001.00,2008.00,2013.00
Yield,28242.0,77053.332094,84956.612897,50.00,19919.2500,38295.00,104676.75,501412.00
Avg_rainfall_mm_per_year,28242.0,1149.055980,709.812150,51.00,593.0000,1083.00,1668.00,3240.00
Pesticide_Tons,28242.0,37076.909344,59958.784665,0.04,1702.0000,17529.44,48687.88,367778.00
Avg_temp,28242.0,20.542627,6.312051,1.30,16.7025,21.51,26.00,30.65


In [47]:
yield_df['Item'].value_counts()

Item
Potatoes                4276
Maize                   4121
Wheat                   3857
Rice, paddy             3388
Soybeans                3223
Sorghum                 3039
Sweet potatoes          2890
Cassava                 2045
Yams                     847
Plantains and others     556
Name: count, dtype: int64

In [48]:
yield_df_backup = yield_df

In [49]:
yield_df_backup.head()

,ID,Location,Item,Year,Yield,Avg_rainfall_mm_per_year,Pesticide_Tons,Avg_temp
0,0,Albania,Maize,1990,36613,1485.0,121.0,16.37
1,1,Albania,Potatoes,1990,66667,1485.0,121.0,16.37
2,2,Albania,"Rice, paddy",1990,23333,1485.0,121.0,16.37
3,3,Albania,Sorghum,1990,12500,1485.0,121.0,16.37
4,4,Albania,Soybeans,1990,7000,1485.0,121.0,16.37


In [50]:
yield_df_backup_copy = yield_df_backup.drop(columns=['ID'])

In [51]:
##dropping missing values
yield_df_ml = yield_df_backup_copy.dropna(axis=0)

In [52]:
yield_df_ml.head()

,Location,Item,Year,Yield,Avg_rainfall_mm_per_year,Pesticide_Tons,Avg_temp
0,Albania,Maize,1990,36613,1485.0,121.0,16.37
1,Albania,Potatoes,1990,66667,1485.0,121.0,16.37
2,Albania,"Rice, paddy",1990,23333,1485.0,121.0,16.37
3,Albania,Sorghum,1990,12500,1485.0,121.0,16.37
4,Albania,Soybeans,1990,7000,1485.0,121.0,16.37


In [53]:
y = yield_df_ml['Yield']

In [54]:
X = yield_df_ml[['Location', 'Item', 'Avg_rainfall_mm_per_year', 'Pesticide_Tons', 'Avg_temp']]

In [55]:
numerical = X.drop(['Location','Item'], axis=1)
numerical.head()

,Avg_rainfall_mm_per_year,Pesticide_Tons,Avg_temp
0,1485.0,121.0,16.37
1,1485.0,121.0,16.37
2,1485.0,121.0,16.37
3,1485.0,121.0,16.37
4,1485.0,121.0,16.37


In [56]:
categorical = X.filter(['Location','Item'])
categorical.head()

,Location,Item
0,Albania,Maize
1,Albania,Potatoes
2,Albania,"Rice, paddy"
3,Albania,Sorghum
4,Albania,Soybeans


In [57]:
cat_numerical = pd.get_dummies(categorical,drop_first=True)
cat_numerical.head()

,Location_Algeria,Location_Angola,Location_Argentina,Location_Armenia,Location_Australia,Location_Austria,Location_Azerbaijan,Location_Bahamas,Location_Bahrain,Location_Bangladesh,...,Location_Zimbabwe,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,Item_Soybeans,Item_Sweet potatoes,Item_Wheat,Item_Yams
0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [58]:
X = pd.concat([numerical, cat_numerical], axis = 1)
X.head()

,Avg_rainfall_mm_per_year,Pesticide_Tons,Avg_temp,Location_Algeria,Location_Angola,Location_Argentina,Location_Armenia,Location_Australia,Location_Austria,Location_Azerbaijan,...,Location_Zimbabwe,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,Item_Soybeans,Item_Sweet potatoes,Item_Wheat,Item_Yams
0,1485.0,121.0,16.37,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1485.0,121.0,16.37,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,1485.0,121.0,16.37,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1485.0,121.0,16.37,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,1485.0,121.0,16.37,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [59]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [60]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform (X_test)

In [61]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
regressor = lin_reg.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 29642.090402772646
Mean Squared Error: 1857377727.868841
Root Mean Squared Error: 43097.305343476415


In [62]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(random_state=42, n_estimators=500)
regressor = rf_reg.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 5561.1498792665925
Mean Squared Error: 172558049.04260904
Root Mean Squared Error: 13136.135239963429


In [63]:
from sklearn.linear_model import LinearRegression

rf_reg = RandomForestRegressor(random_state=42, n_estimators=500)
regressor = rf_reg.fit(X_train, y_train)

single_record = sc.transform (X.values[19].reshape(1, -1))
predicted_mpg = regressor.predict(single_record)
print(predicted_mpg)

[94841.602]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



In [64]:
yield_df_ml.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,28242.0,2001.544296,7.051905,1990.00,1995.0000,2001.00,2008.00,2013.00
Yield,28242.0,77053.332094,84956.612897,50.00,19919.2500,38295.00,104676.75,501412.00
Avg_rainfall_mm_per_year,28242.0,1149.055980,709.812150,51.00,593.0000,1083.00,1668.00,3240.00
Pesticide_Tons,28242.0,37076.909344,59958.784665,0.04,1702.0000,17529.44,48687.88,367778.00
Avg_temp,28242.0,20.542627,6.312051,1.30,16.7025,21.51,26.00,30.65


In [65]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state=42, n_estimators=500)
regressor = rf_reg.fit(X_train, y_train)

predictions = regressor.predict(X_test)

print("The yield predictions for 10 rows are")
print(predictions[:10].round(2))
print("Comparing the average of the predictions to average of the original (77053.33) helps confirm the predictions")
print(np.mean(predictions))
print("Here we can see the accuracy of the model, it  is almost 100%")
regressor.score(X_train, y_train)

The yield predictions for 10 rows are
[ 24704.61  99535.77  37179.85  30473.96  15449.39 169671.11  13259.47
  42298.96 182656.22 106147.47]
Comparing the average of the predictions to average of the original (77053.33) helps confirm the predictions
78063.71769027066
Here we can see the accuracy of the model, it  is almost 100%


0.9963912471783498

In [66]:
import numpy as np
# Function to calculate Mean Squared Error (MSE)
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def predict_yield(item, year, avg_rainfall, pesticide_tons, avg_temp):

    predicted_yield = 2.5 * year + 1.5 * avg_rainfall - 0.5 * pesticide_tons + 10 * avg_temp

    return predicted_yield

# Get input from the user
item = input("Item: ")
year = int(input("Year: "))
avg_rainfall = float(input("Avg_rainfall_mm_per_year: "))
pesticide_tons = float(input("Pesticide_Tons: "))
avg_temp = float(input("Avg_temp: "))

# Make prediction
predicted_yield = predict_yield(item, year, avg_rainfall, pesticide_tons, avg_temp)

# Print the predicted yield
print("Predicted yield:", predicted_yield)
actual_yield = 50  # Actual yield
mse = mean_squared_error(actual_yield, predicted_yield)
print("Mean Squared Error:", mse)


Item: maize
Year: 2023
Avg_rainfall_mm_per_year: 6532
Pesticide_Tons: 261
Avg_temp: 25
Predicted yield: 14975.0
Mean Squared Error: 222755625.0
